In [ ]:
1

In [ ]:
using Pkg
Pkg.activate("..")
using Pun
using Plots

include("../examples.jl")

In [ ]:
struct LinearModel
    slope::Real
    intercept::Real
end

linear_model_prior() = @prob begin
    slope <<= normal(0, 1)
    intercept <<= normal(0, 2)
    return LinearModel(slope, intercept)
end

bayesian_linear_regression(xs::Vector{Float64}) = @prob begin
    linear_model <<= linear_model_prior()
    
    # Given the slope and intercept, we can sample y coordinates
    # for each of the x coordinates in our input vector.
    y <<= mapM(x -> normal(linear_model.slope * x + linear_model.intercept, 0.1), xs)
    linear_model >>= @prob begin
        slope <<= normal(inv(transpose(xs) * xs) * (transpose(xs) * y), 1)
        intercept <<= normal(0, 2)
        return LinearModel(slope, intercept)
    end
    return y
end

In [ ]:
xs = [-5., -4., -3., -2., -1., 0., 1., 2., 3., 4., 5.]
y = [dual.value for dual in simulate(bayesian_linear_regression(xs))[1]]

plot(xs, y, seriestype=:scatter, label="Observed Data", title="Bayesian Linear Regression", xlabel="x", ylabel="y")

In [ ]:
model = bayesian_linear_regression(xs)
proposal = y -> model # We use the prior as the proposal distribution

importance_sampling(model, proposal, y, 1000)

In [ ]:
s.logweight